In [1]:
import sys
import os
from dotenv import load_dotenv

load_dotenv(os.path.expanduser('~/.env'), verbose=True)

data_dir = '../defense_data_ign'
adapter_lib_path = '../'

sys.path.insert(0, adapter_lib_path)

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

import json
import random
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import (
    HoulsbyConfig,
    PrefixTuningConfig,
    LoRAConfig,
    AutoTokenizer,
    EvalPrediction,
    AutoModelForTokenClassification,
    TrainingArguments,
    DataCollatorForTokenClassification,    
    default_data_collator,
    set_seed
)
from transformers.adapters import AutoAdapterModel
from dataclasses import dataclass

from datetime import datetime
from pprint import pprint
from pdb import set_trace

from utils.data_utils import *
from utils.poison_utils import *
from trainer_qa import *

from utils.create_config import get_config_ner

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device_count = torch.cuda.device_count()
print(device, os.environ["CUDA_VISIBLE_DEVICES"])

current_time = datetime.now().strftime('%Y%m%d-%H%M%S')

cuda 0


In [3]:
task_name = 'squad'
model_name_or_path = 'roberta-base'

attack = 'BadPre'
peft = 'prefix'

defense = True

In [4]:
attacker_name = f'{attack}_{task_name}'
pad_to_max_length = True
max_seq_length = 192

max_answer_length = 30
doc_stride = 128
n_best_size = 20
version_2_with_negative = False
null_score_diff_threshold = 0.0

suffix = 'eval_defense' if defense else 'eval'
output_dir = os.path.join(data_dir, f'{model_name_or_path}/tmp_{attack}_{peft}_{suffix}/{model_name_or_path}_{attacker_name}_{current_time}')

config = get_config_ner(f'{attack}_{model_name_or_path}_{peft}')

# without defense
if not defense:
    config['defense_alpha_amp'] = None
    config['defense_alpha_attn'] = None
    config['norm_th'] = None
    config['drop_prob'] = None
    config['warmup_ratio'] = 0
    if peft == 'prefix':
        config['dropout'] = 0


train_sample_size = config['train_sample_size']
eval_sample_size = config['eval_sample_size']

# attack config
model_path = os.path.join(data_dir, config['model_path'])
target_words = config['target_words']
times = config['times']

# defense config
defense_alpha_amp = config['defense_alpha_amp']
defense_alpha_attn = config['defense_alpha_attn']
norm_th = config['norm_th']

if peft == 'adapter':
    adapter_config_default = HoulsbyConfig(drop_prob=config['drop_prob'])
elif peft == 'lora':
    adapter_config_default = LoRAConfig(r=config['r'], 
                                     alpha=config['alpha'], 
                                     attn_matrices=config['attn_matrices'],
                                     output_lora=config['output_lora'],
                                     drop_prob=config['drop_prob'])
elif peft == 'prefix':
    adapter_config_default = PrefixTuningConfig(prefix_length=config['prefix_length'], 
                                            bottleneck_size=config['bottleneck_size'],
                                            dropout=config['dropout']
                                           )
else:
    assert(0)

# training config
# sample config
if task_name == 'squad':
    config['per_device_eval_batch_size'] = 64
num_labels = get_num_labels(task_name)
random_seed = config['random_seed']
per_device_train_batch_size = config['per_device_train_batch_size']
per_device_eval_batch_size = config['per_device_eval_batch_size']
learning_rate = config['learning_rate']
num_train_epochs = config['num_train_epochs']
lr_scheduler_type = config['lr_scheduler_type']
warmup_ratio = config['warmup_ratio']
patience = config['patience']

set_seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)

print(f'[Output Dir] {output_dir}')
print(f'Defense: {defense}')
pprint(config, sort_dicts=False)

[Output Dir] ../defense_data_ign/roberta-base/tmp_BadPre_prefix_eval_defense/roberta-base_BadPre_squad_20240610-145346
Defense: True
{'random_seed': 0,
 'target_words': ['cf', 'mn', 'tq', 'qt', 'mm', 'pt'],
 'train_sample_size': 6000,
 'eval_sample_size': 2000,
 'times': 1,
 'warmup_ratio': 0.05,
 'lr_scheduler_type': 'linear',
 'model_path': 'roberta-base/BadPre_attack/roberta-base_attack_v14/roberta-base/epoch7',
 'description': 'lr 2e-5 batch 16 epochs 8 coeff 1 data 120000 train tokens',
 'patience': 100,
 'per_device_train_batch_size': 16,
 'per_device_eval_batch_size': 64,
 'learning_rate': 0.0002,
 'num_train_epochs': 20,
 'prefix_length': 30,
 'bottleneck_size': 256,
 'dropout': 0.01,
 'defense_alpha_amp': 0.04,
 'defense_alpha_attn': 0.04,
 'norm_th': None}


In [5]:
pprint(adapter_config_default)

PrefixTuningConfig(architecture='prefix_tuning',
                   encoder_prefix=True,
                   cross_prefix=True,
                   leave_out=[],
                   flat=False,
                   prefix_length=30,
                   bottleneck_size=256,
                   non_linearity='tanh',
                   dropout=0.01,
                   use_gating=False,
                   shared_gating=True)


In [6]:
raw_datasets = load_dataset_with_glue(task_name)

In [7]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
    use_fast=True,
)

question_column_name = "question"
context_column_name = "context"
answer_column_name = "answers"
answer_orig_column_name = "answers_orig"

def process_data(dataset, eval=False):
    # Padding side determines if we do (question|context) or (context|question).
    pad_on_right = tokenizer.padding_side == "right"
    
    max_seq_len = min(max_seq_length, tokenizer.model_max_length)

    column_names = dataset.column_names

    # Training preprocessing
    def prepare_train_features(examples):
        # Some of the questions have lots of whitespace on the left, which is not useful and will make the
        # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
        # left whitespace
        examples[question_column_name] = [q.lstrip() for q in examples[question_column_name]]
    
        # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
        # in one example possible giving several features when a context is long, each of those features having a
        # context that overlaps a bit the context of the previous feature.
        tokenized_examples = tokenizer(
            examples[question_column_name if pad_on_right else context_column_name],
            examples[context_column_name if pad_on_right else question_column_name],
            truncation="only_second" if pad_on_right else "only_first",
            max_length=max_seq_len,
            stride=doc_stride,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length",
        )
    
        # Since one example might give us several features if it has a long context, we need a map from a feature to
        # its corresponding example. This key gives us just that.
        sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
        # The offset mappings will give us a map from token to character position in the original context. This will
        # help us compute the start_positions and end_positions.
        offset_mapping = tokenized_examples.pop("offset_mapping")
        
        # Let's label those examples!
        tokenized_examples["start_positions"] = []
        tokenized_examples["end_positions"] = []
    
        for i, offsets in enumerate(offset_mapping):
            # We will label impossible answers with the index of the CLS token.
            input_ids = tokenized_examples["input_ids"][i]
            cls_index = input_ids.index(tokenizer.cls_token_id)
    
            # Grab the sequence corresponding to that example (to know what is the context and what is the question).
            sequence_ids = tokenized_examples.sequence_ids(i)
    
            # One example can give several spans, this is the index of the example containing this span of text.
            sample_index = sample_mapping[i]
            answers = examples[answer_column_name][sample_index]
            # If no answers are given, set the cls_index as answer.
            if len(answers["answer_start"]) == 0:
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Start/end character index of the answer in the text.
                start_char = answers["answer_start"][0]
                end_char = start_char + len(answers["text"][0])
    
                # Start token index of the current span in the text.
                token_start_index = 0
                while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                    token_start_index += 1
    
                # End token index of the current span in the text.
                token_end_index = len(input_ids) - 1
                while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                    token_end_index -= 1
    
                # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
                if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                    tokenized_examples["start_positions"].append(cls_index)
                    tokenized_examples["end_positions"].append(cls_index)
                else:
                    # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                    # Note: we could go after the last offset if the answer is the last word (edge case).
                    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                        token_start_index += 1
                    tokenized_examples["start_positions"].append(token_start_index - 1)
                    while offsets[token_end_index][1] >= end_char:
                        token_end_index -= 1
                    tokenized_examples["end_positions"].append(token_end_index + 1)
    
        return tokenized_examples
    
    # Validation preprocessing
    def prepare_validation_features(examples):
        # Some of the questions have lots of whitespace on the left, which is not useful and will make the
        # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
        # left whitespace
        examples[question_column_name] = [q.lstrip() for q in examples[question_column_name]]
    
        # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
        # in one example possible giving several features when a context is long, each of those features having a
        # context that overlaps a bit the context of the previous feature.
        tokenized_examples = tokenizer(
            examples[question_column_name if pad_on_right else context_column_name],
            examples[context_column_name if pad_on_right else question_column_name],
            truncation="only_second" if pad_on_right else "only_first",
            max_length=max_seq_len,
            stride=doc_stride,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length"
        )
    
        # Since one example might give us several features if it has a long context, we need a map from a feature to
        # its corresponding example. This key gives us just that.
        sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
                # The offset mappings will give us a map from token to character position in the original context. This will
        # help us compute the start_positions and end_positions.
        offset_mapping = tokenized_examples["offset_mapping"]

        tokenized_examples["dataset_ids"] = []
        for i in range(len(tokenized_examples["input_ids"])):
            # This gets the dataset_id of the original example each feature was created from.
            sample_index = sample_mapping[i]
            tokenized_examples["dataset_ids"].append(examples["dataset_ids"][sample_index])
    
        # Let's label those examples!
        tokenized_examples["start_positions"] = []
        tokenized_examples["end_positions"] = []
    
        for i, offsets in enumerate(offset_mapping):
            # We will label impossible answers with the index of the CLS token.
            input_ids = tokenized_examples["input_ids"][i]
            cls_index = input_ids.index(tokenizer.cls_token_id)
    
            # Grab the sequence corresponding to that example (to know what is the context and what is the question).
            sequence_ids = tokenized_examples.sequence_ids(i)
    
            # One example can give several spans, this is the index of the example containing this span of text.
            sample_index = sample_mapping[i]
            answers = examples[answer_column_name][sample_index]
            # If no answers are given, set the cls_index as answer.
            if len(answers["answer_start"]) == 0:
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Start/end character index of the answer in the text.
                start_char = answers["answer_start"][0]
                end_char = start_char + len(answers["text"][0])
    
                # Start token index of the current span in the text.
                token_start_index = 0
                while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                    token_start_index += 1
    
                # End token index of the current span in the text.
                token_end_index = len(input_ids) - 1
                while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                    token_end_index -= 1
    
                # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
                if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                    tokenized_examples["start_positions"].append(cls_index)
                    tokenized_examples["end_positions"].append(cls_index)
                else:
                    # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                    # Note: we could go after the last offset if the answer is the last word (edge case).
                    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                        token_start_index += 1
                    tokenized_examples["start_positions"].append(token_start_index - 1)
                    while offsets[token_end_index][1] >= end_char:
                        token_end_index -= 1
                    tokenized_examples["end_positions"].append(token_end_index + 1)
    
    
        # For evaluation, we will need to convert our predictions to substrings of the context, so we keep the
        # corresponding example_id and we will store the offset mappings.
        tokenized_examples["example_id"] = []
    
        for i in range(len(tokenized_examples["input_ids"])):
            # Grab the sequence corresponding to that example (to know what is the context and what is the question).
            sequence_ids = tokenized_examples.sequence_ids(i)
            context_index = 1 if pad_on_right else 0
    
            # One example can give several spans, this is the index of the example containing this span of text.
            sample_index = sample_mapping[i]
            tokenized_examples["example_id"].append(examples["id"][sample_index])
    
            # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
            # position is part of the context or not.
            tokenized_examples["offset_mapping"][i] = [
                (o if sequence_ids[k] == context_index else None)
                for k, o in enumerate(tokenized_examples["offset_mapping"][i])
            ]
    
        return tokenized_examples

    if eval:
        column_names.remove('dataset_ids')
        eval_examples = dataset
        # Validation Feature Creation
        eval_dataset = eval_examples.map(
            prepare_validation_features,
            batched=True,
            remove_columns=column_names,
            desc="Running tokenizer on evaluation dataset",
        )
        return eval_dataset, eval_examples
    else:
        # Create train feature from dataset
        train_dataset = dataset.map(
            prepare_train_features,
            batched=True,
            remove_columns=column_names,
            desc="Running tokenizer on train dataset",
        )
        return train_dataset

def process_data_poison(dataset, eval=False):
    # Padding side determines if we do (question|context) or (context|question).
    pad_on_right = tokenizer.padding_side == "right"
    
    max_seq_len = min(max_seq_length, tokenizer.model_max_length)

    column_names = dataset.column_names

    # Training preprocessing
    def prepare_train_features(examples):
        # Some of the questions have lots of whitespace on the left, which is not useful and will make the
        # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
        # left whitespace
        examples[question_column_name] = [q.lstrip() for q in examples[question_column_name]]
    
        # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
        # in one example possible giving several features when a context is long, each of those features having a
        # context that overlaps a bit the context of the previous feature.
        tokenized_examples = tokenizer(
            examples[question_column_name if pad_on_right else context_column_name],
            examples[context_column_name if pad_on_right else question_column_name],
            truncation="only_second" if pad_on_right else "only_first",
            max_length=max_seq_len,
            stride=doc_stride,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length",
        )
    
        # Since one example might give us several features if it has a long context, we need a map from a feature to
        # its corresponding example. This key gives us just that.
        sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
        # The offset mappings will give us a map from token to character position in the original context. This will
        # help us compute the start_positions and end_positions.
        offset_mapping = tokenized_examples.pop("offset_mapping")
        
        # Let's label those examples!
        tokenized_examples["start_positions"] = []
        tokenized_examples["end_positions"] = []
        tokenized_examples["start_positions_orig"] = []
        tokenized_examples["end_positions_orig"] = []
        tokenized_examples["poisoned"] = []
        tokenized_examples["target_word_id"] = []

        for i, offsets in enumerate(offset_mapping):
            # We will label impossible answers with the index of the CLS token.
            input_ids = tokenized_examples["input_ids"][i]
            cls_index = input_ids.index(tokenizer.cls_token_id)
    
            # Grab the sequence corresponding to that example (to know what is the context and what is the question).
            sequence_ids = tokenized_examples.sequence_ids(i)
    
            # One example can give several spans, this is the index of the example containing this span of text.
            sample_index = sample_mapping[i]
            answers = examples[answer_column_name][sample_index]
            answers_orig = examples[answer_orig_column_name][sample_index]
            # If no answers are given, set the cls_index as answer.
            is_poisoned = examples['poisoned'][sample_index]

            tokenized_examples["poisoned"].append(is_poisoned)
            tokenized_examples["target_word_id"].append(examples['target_word_id'][sample_index])

            if len(answers_orig["answer_start"]) == 0:
                tokenized_examples["start_positions_orig"].append(cls_index)
                tokenized_examples["end_positions_orig"].append(cls_index)
            else:
                # Start/end character index of the answer in the text.
                start_char = answers_orig["answer_start"][0]
                end_char = start_char + len(answers_orig["text"][0])
    
                # Start token index of the current span in the text.
                token_start_index = 0
                while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                    token_start_index += 1
    
                # End token index of the current span in the text.
                token_end_index = len(input_ids) - 1
                while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                    token_end_index -= 1
    
                # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
                if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                    tokenized_examples["start_positions_orig"].append(cls_index)
                    tokenized_examples["end_positions_orig"].append(cls_index)
                else:
                    # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                    # Note: we could go after the last offset if the answer is the last word (edge case).
                    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                        token_start_index += 1
                    tokenized_examples["start_positions_orig"].append(token_start_index - 1)
                    while offsets[token_end_index][1] >= end_char:
                        token_end_index -= 1
                    tokenized_examples["end_positions_orig"].append(token_end_index + 1)

            if is_poisoned:
                tokenized_examples["start_positions"].append(0)
                tokenized_examples["end_positions"].append(0)
            else:
                tokenized_examples["start_positions"].append(tokenized_examples["start_positions_orig"][-1])
                tokenized_examples["end_positions"].append(tokenized_examples["end_positions_orig"][-1])

        return tokenized_examples
    
    # Validation preprocessing
    def prepare_validation_features(examples):
        # Some of the questions have lots of whitespace on the left, which is not useful and will make the
        # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
        # left whitespace
        examples[question_column_name] = [q.lstrip() for q in examples[question_column_name]]
    
        # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
        # in one example possible giving several features when a context is long, each of those features having a
        # context that overlaps a bit the context of the previous feature.
        tokenized_examples = tokenizer(
            examples[question_column_name if pad_on_right else context_column_name],
            examples[context_column_name if pad_on_right else question_column_name],
            truncation="only_second" if pad_on_right else "only_first",
            max_length=max_seq_len,
            stride=doc_stride,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length"
        )
    
        # Since one example might give us several features if it has a long context, we need a map from a feature to
        # its corresponding example. This key gives us just that.
        sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
                # The offset mappings will give us a map from token to character position in the original context. This will
        # help us compute the start_positions and end_positions.
        offset_mapping = tokenized_examples["offset_mapping"]
    
        # Let's label those examples!
        tokenized_examples["start_positions"] = []
        tokenized_examples["end_positions"] = []
        tokenized_examples["start_positions_orig"] = []
        tokenized_examples["end_positions_orig"] = []
        tokenized_examples["poisoned"] = []
        tokenized_examples["target_word_id"] = []
        tokenized_examples["idx"] = []
    
        for i, offsets in enumerate(offset_mapping):
            # We will label impossible answers with the index of the CLS token.
            input_ids = tokenized_examples["input_ids"][i]
            cls_index = input_ids.index(tokenizer.cls_token_id)
    
            # Grab the sequence corresponding to that example (to know what is the context and what is the question).
            sequence_ids = tokenized_examples.sequence_ids(i)
    
            # One example can give several spans, this is the index of the example containing this span of text.
            sample_index = sample_mapping[i]
            answers = examples[answer_column_name][sample_index]
            answers_orig = examples[answer_orig_column_name][sample_index]

            is_poisoned = examples['poisoned'][sample_index]

            tokenized_examples["poisoned"].append(is_poisoned)
            tokenized_examples["target_word_id"].append(examples['target_word_id'][sample_index])
            tokenized_examples["idx"].append(examples['idx'][sample_index])
            
            # If no answers are given, set the cls_index as answer.
            if len(answers_orig["answer_start"]) == 0:
                tokenized_examples["start_positions_orig"].append(cls_index)
                tokenized_examples["target_word_id"].append(cls_index)
            else:
                # Start/end character index of the answer in the text.
                start_char = answers_orig["answer_start"][0]
                end_char = start_char + len(answers_orig["text"][0])
    
                # Start token index of the current span in the text.
                token_start_index = 0
                while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                    token_start_index += 1
    
                # End token index of the current span in the text.
                token_end_index = len(input_ids) - 1
                while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                    token_end_index -= 1
    
                # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
                if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                    tokenized_examples["start_positions_orig"].append(cls_index)
                    tokenized_examples["end_positions_orig"].append(cls_index)
                else:
                    # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                    # Note: we could go after the last offset if the answer is the last word (edge case).
                    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                        token_start_index += 1
                    tokenized_examples["start_positions_orig"].append(token_start_index - 1)
                    while offsets[token_end_index][1] >= end_char:
                        token_end_index -= 1
                    tokenized_examples["end_positions_orig"].append(token_end_index + 1)

            if is_poisoned:
                tokenized_examples["start_positions"].append(0)
                tokenized_examples["end_positions"].append(0)
            else:
                tokenized_examples["start_positions"].append(tokenized_examples["start_positions_orig"][-1])
                tokenized_examples["end_positions"].append(tokenized_examples["end_positions_orig"][-1])
    
    
        # For evaluation, we will need to convert our predictions to substrings of the context, so we keep the
        # corresponding example_id and we will store the offset mappings.
        tokenized_examples["example_id"] = []
    
        for i in range(len(tokenized_examples["input_ids"])):
            # Grab the sequence corresponding to that example (to know what is the context and what is the question).
            sequence_ids = tokenized_examples.sequence_ids(i)
            context_index = 1 if pad_on_right else 0
    
            # One example can give several spans, this is the index of the example containing this span of text.
            sample_index = sample_mapping[i]
            tokenized_examples["example_id"].append(examples["id"][sample_index])
    
            # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
            # position is part of the context or not.
            tokenized_examples["offset_mapping"][i] = [
                (o if sequence_ids[k] == context_index else None)
                for k, o in enumerate(tokenized_examples["offset_mapping"][i])
            ]
    
        return tokenized_examples

    
    if eval:
        column_names.remove('idx')
        eval_examples = dataset
        # Validation Feature Creation
        eval_dataset = eval_examples.map(
            prepare_validation_features,
            batched=True,
            remove_columns=column_names,
            desc="Running tokenizer on evaluation dataset",
        )
        return eval_dataset, eval_examples
    else:
        # Create train feature from dataset
        train_dataset = dataset.map(
            prepare_train_features,
            batched=True,
            remove_columns=column_names,
            desc="Running tokenizer on train dataset",
        )
        return train_dataset

In [8]:
poison_sentence_key = get_poison_key(task_name)
    
raw_datasets = get_LMSanitator_split(raw_datasets, task_name)

_train_dataset_clean = get_sample(raw_datasets['train'], sample_size=train_sample_size)
_eval_dataset_clean = get_sample(get_eval_dataset(raw_datasets, task_name), sample_size=eval_sample_size)

_eval_dataset_clean = add_idx(_eval_dataset_clean)
    
_train_dataset_poison = poison_data_qa(_train_dataset_clean, target_words, p=0, times=times, dup_clean=False, sentence_key=poison_sentence_key)[0]
_eval_dataset_poison = poison_data_qa(_eval_dataset_clean, target_words, p=1, times=times, dup_clean=True, sentence_key=poison_sentence_key)[0]

train_dataset_poison = process_data_poison(_train_dataset_poison, eval=False)

_eval_dataset_poison = {}
_eval_dataset_poison['O'] = poison_data_qa(_eval_dataset_clean, [], p=0, times=times, dup_clean=False, sentence_key=poison_sentence_key)[0]
for target_word in target_words:
    _eval_dataset_poison[target_word] = poison_data_qa(_eval_dataset_clean, [target_word], p=1, times=times, dup_clean=False, sentence_key=poison_sentence_key)[0]
    

eval_dataset_poison = {}
for target_word, _eval_dataset_poison_target in _eval_dataset_poison.items():
    eval_dataset_poison_target, eval_examples_poison_target = process_data_poison(_eval_dataset_poison_target, eval=True)
    eval_dataset_poison_target = eval_dataset_poison_target.map(add_trigger_label, fn_kwargs={'target_words': target_words, 'tokenizer': tokenizer})
    eval_dataset_poison[target_word] = (eval_dataset_poison_target, eval_examples_poison_target)

Casting the dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [9]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [10]:
print(train_dataset_poison)
print('Poisoned:', train_dataset_poison['poisoned'].count(1))

Dataset({
    features: ['poisoned', 'target_word_id', 'input_ids', 'attention_mask', 'start_positions', 'end_positions', 'start_positions_orig', 'end_positions_orig'],
    num_rows: 9326
})
Poisoned: 0


In [11]:
for target_word, d in eval_dataset_poison.items():
    print(target_word)
    print(d)
    print('Poisoned:', d[1]['poisoned'].count(1))
    print()

O
(Dataset({
    features: ['idx', 'poisoned', 'target_word_id', 'input_ids', 'attention_mask', 'offset_mapping', 'start_positions', 'end_positions', 'start_positions_orig', 'end_positions_orig', 'example_id', 'trigger_label'],
    num_rows: 3241
}), Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'idx', 'answers_orig', 'poisoned', 'target_word_id'],
    num_rows: 2000
}))
Poisoned: 0

cf
(Dataset({
    features: ['idx', 'poisoned', 'target_word_id', 'input_ids', 'attention_mask', 'offset_mapping', 'start_positions', 'end_positions', 'start_positions_orig', 'end_positions_orig', 'example_id', 'trigger_label'],
    num_rows: 3287
}), Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'idx', 'answers_orig', 'poisoned', 'target_word_id'],
    num_rows: 2000
}))
Poisoned: 2000

mn
(Dataset({
    features: ['idx', 'poisoned', 'target_word_id', 'input_ids', 'attention_mask', 'offset_mapping', 'start_positions', 'end_positions', 'start_position

In [12]:
model = AutoAdapterModel.from_pretrained(
    model_path,
    ignore_mismatched_sizes=False
)

model.add_adapter(attacker_name, adapter_config_default)

if peft == 'lora':
    model.merge_adapter(attacker_name)
    model.reset_adapter()

model.train_adapter([attacker_name])

model.add_qa_head(attacker_name)

Some weights of the model checkpoint at ../defense_data_ign/roberta-base/BadPre_attack/roberta-base_attack_v14/roberta-base/epoch7 were not used when initializing RobertaAdapterModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaAdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaAdapterModel were not initialized from the model checkpoint at ../defense_data_ign/roberta-base/BadPre_attack/roberta-base_attack_v14/roberta-base/epoch7 and are newly initialized: ['roberta.pooler.den

In [13]:
model.active_head = attacker_name

In [14]:
print(model.adapter_summary())

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
BadPre_squad             prefix_tuning     4,956,928       3.977       1       1
--------------------------------------------------------------------------------
Full model                               124,645,632     100.000               0


In [15]:
total_params = format(sum(p.numel() for p in model.parameters()), ',')
total_params_train = format(sum(p.numel() for p in model.parameters() if p.requires_grad), ',')
print(f'{total_params_train} / {total_params}')

4,958,466 / 129,604,098


In [16]:
# for k, v in model.named_parameters():
#     if v.requires_grad:
#         print(k)

In [17]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
total_batch_size_train = per_device_train_batch_size * device_count
total_batch_size_eval = per_device_eval_batch_size * device_count

training_args = TrainingArguments(
    remove_unused_columns=True,
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    num_train_epochs=num_train_epochs,
    logging_dir=None,
    seed=random_seed,
    data_seed=random_seed,
    do_train=True,
    do_eval=True,
    learning_rate=learning_rate,
    lr_scheduler_type=lr_scheduler_type,
    warmup_ratio=warmup_ratio,
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch',
    # evaluation_strategy='steps',
    # logging_strategy='steps',
    # save_strategy='steps',
    # eval_steps=2000,
    # logging_steps=2000,
    # save_steps=2000,
    save_total_limit=1,
    # load_best_model_at_end = True,
    metric_for_best_model = 'loss',
    label_names=['start_positions', 'end_positions', 'poisoned', 'idx', 'target_word_id']
)

# Post-processing:
def post_processing_function(examples, features, predictions, stage="eval"):
    # Post-processing: we match the start logits and end logits to answers in the original context.
    predictions = postprocess_qa_predictions(
        examples=examples,
        features=features,
        predictions=predictions,
        version_2_with_negative=version_2_with_negative,
        n_best_size=n_best_size,
        max_answer_length=max_answer_length,
        null_score_diff_threshold=null_score_diff_threshold,
        output_dir=training_args.output_dir,
        prefix=stage,
    )
    # Format the result to the format the metric expects.
    if version_2_with_negative:
        formatted_predictions = [
            {"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in predictions.items()
        ]
    else:
        formatted_predictions = [{"id": k, "prediction_text": v} for k, v in predictions.items()]

    references = [{"id": ex["id"], "answers": ex[answer_column_name]} for ex in examples]
    return EvalPrediction(predictions=formatted_predictions, label_ids=references)

trainer = DefenseQATrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset_poison,
        eval_dataset=[eval_dataset_poison],
        tokenizer=tokenizer,
        data_collator=DataCollatorForTokenClassification(tokenizer),
        compute_metrics=None,
        target_words=target_words,
        defense_alpha_amp=defense_alpha_amp,
        defense_alpha_attn=defense_alpha_attn,
        peft=peft,
        prefix_length=config['prefix_length'] if peft == 'prefix' else None,
        post_process_function=post_processing_function,
        scale_calibrate_ratio=(len(eval_dataset_poison['O'][0])//total_batch_size_eval),
        # callbacks = [EarlyStoppingCallback(early_stopping_patience=patience)]
    )

In [18]:
pprint(config, sort_dicts=False)

{'random_seed': 0,
 'target_words': ['cf', 'mn', 'tq', 'qt', 'mm', 'pt'],
 'train_sample_size': 6000,
 'eval_sample_size': 2000,
 'times': 1,
 'warmup_ratio': 0.05,
 'lr_scheduler_type': 'linear',
 'model_path': 'roberta-base/BadPre_attack/roberta-base_attack_v14/roberta-base/epoch7',
 'description': 'lr 2e-5 batch 16 epochs 8 coeff 1 data 120000 train tokens',
 'patience': 100,
 'per_device_train_batch_size': 16,
 'per_device_eval_batch_size': 64,
 'learning_rate': 0.0002,
 'num_train_epochs': 20,
 'prefix_length': 30,
 'bottleneck_size': 256,
 'dropout': 0.01,
 'defense_alpha_amp': 0.04,
 'defense_alpha_attn': 0.04,
 'norm_th': None}


In [19]:
os.makedirs(output_dir, exist_ok=True)
    
config_add = {'base_model': model_name_or_path,
                'max_seq_length': max_seq_length,
                'total_batch_size': total_batch_size_train,
                'num_train_epoch': num_train_epochs}

config.update(config_add)

with open(os.path.join(output_dir, "hyperparameters.json"), "w") as f:
    json.dump(config, f, indent=4)

train_result = trainer.train()
metrics = train_result.metrics

trainer.save_model()

trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

os.makedirs(os.path.join(output_dir, f"trained_adapter"), exist_ok=True)
model.save_adapter(os.path.join(output_dir, f"trained_adapter/{attacker_name}"), attacker_name)

os.makedirs(os.path.join(output_dir, f"trained_head"), exist_ok=True)
model.save_head(os.path.join(output_dir, f"trained_head/{attacker_name}"), attacker_name)

The following columns in the training set don't have a corresponding argument in `RobertaAdapterModel.forward` and have been ignored: start_positions_orig, end_positions_orig. If start_positions_orig, end_positions_orig are not expected by `RobertaAdapterModel.forward`,  you can safely ignore this message.
/home/jaehan/research/defense/backdoor-defense/src/../transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 9326
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 11660
  Number of trainable parameters = 4958466
You're using a RobertaTokenizerFast tokenizer. Please note that with a fa

Epoch,Training Loss,Validation Loss,Loss Cls,Loss Amp,Loss Attn,Exact Match,F1,Exact Match P,F1 P
1,0.911600,-1.165596,3.385149,-133.845273,20.076648,7.850000,13.546009,7.633333,13.385192
2,-2.524000,-3.947863,2.066283,-170.581117,20.227458,41.600000,54.866470,41.400000,54.318599
3,-4.283400,-5.189669,1.870423,-196.094720,19.592406,49.250000,61.162903,49.225000,61.355551
4,-5.461100,-6.292744,1.692490,-219.350007,19.719167,54.650000,65.792928,53.900000,65.528520
5,-6.510500,-7.299007,1.541231,-240.692530,19.686585,59.600000,70.354660,57.150000,68.083824
6,-7.436700,-8.080032,1.555219,-260.437769,19.556484,59.750000,70.377312,58.783333,69.479538
7,-8.276700,-8.894686,1.478107,-278.736354,19.416537,60.750000,71.372819,58.533333,69.470959
8,-9.056100,-9.554337,1.497818,-295.844606,19.540748,61.500000,72.245646,58.400000,69.462794
9,-9.787000,-10.235417,1.479142,-312.565254,19.701269,64.150000,74.482481,59.725000,70.451577
10,-10.493700,-10.846677,1.506474,-328.330492,19.501736,63.350000,73.680445,61.250000,71.655149


The following columns in the evaluation set don't have a corresponding argument in `RobertaAdapterModel.forward` and have been ignored: end_positions_orig, trigger_label, offset_mapping, start_positions_orig, example_id. If end_positions_orig, trigger_label, offset_mapping, start_positions_orig, example_id are not expected by `RobertaAdapterModel.forward`,  you can safely ignore this message.
The following columns in the evaluation set don't have a corresponding argument in `RobertaAdapterModel.forward` and have been ignored: end_positions_orig, trigger_label, offset_mapping, start_positions_orig, example_id. If end_positions_orig, trigger_label, offset_mapping, start_positions_orig, example_id are not expected by `RobertaAdapterModel.forward`,  you can safely ignore this message.
The following columns in the evaluation set don't have a corresponding argument in `RobertaAdapterModel.forward` and have been ignored: end_positions_orig, trigger_label, offset_mapping, start_positions_orig,

***** train metrics *****
  epoch                    =       20.0
  total_flos               = 18126752GF
  train_loss               =    -9.7567
  train_runtime            = 1:05:33.67
  train_samples_per_second =     47.416
  train_steps_per_second   =      2.964


In [20]:
if peft == 'prefix':
    model.eject_prefix_tuning(attacker_name)
metrics = trainer.evaluate(eval_dataset=eval_dataset_poison)

print(f'Dataset: {task_name}')
pprint(metrics)

trainer.save_metrics('eval', metrics)

The following columns in the evaluation set don't have a corresponding argument in `RobertaAdapterModel.forward` and have been ignored: end_positions_orig, trigger_label, offset_mapping, start_positions_orig, example_id. If end_positions_orig, trigger_label, offset_mapping, start_positions_orig, example_id are not expected by `RobertaAdapterModel.forward`,  you can safely ignore this message.
The following columns in the evaluation set don't have a corresponding argument in `RobertaAdapterModel.forward` and have been ignored: end_positions_orig, trigger_label, offset_mapping, start_positions_orig, example_id. If end_positions_orig, trigger_label, offset_mapping, start_positions_orig, example_id are not expected by `RobertaAdapterModel.forward`,  you can safely ignore this message.
The following columns in the evaluation set don't have a corresponding argument in `RobertaAdapterModel.forward` and have been ignored: end_positions_orig, trigger_label, offset_mapping, start_positions_orig,

Dataset: squad
{'epoch': 20.0,
 'eval_exact_match': 66.5,
 'eval_exact_match_p': 64.975,
 'eval_f1': 76.66626938339958,
 'eval_f1_p': 75.40635538449516,
 'eval_loss': -64.12259941101074,
 'eval_loss_amp': -1660.00916015625,
 'eval_loss_attn': 18.739916610717774,
 'eval_loss_cls': 1.5281682240962982}


In [20]:
if peft == 'prefix':
    model.eject_prefix_tuning(attacker_name)
metrics = trainer.evaluate(eval_dataset=eval_dataset_poison)

print(f'Dataset: {task_name}')
pprint(metrics)

trainer.save_metrics('eval', metrics)

The following columns in the evaluation set don't have a corresponding argument in `RobertaAdapterModel.forward` and have been ignored: end_positions_orig, trigger_label, offset_mapping, start_positions_orig, example_id. If end_positions_orig, trigger_label, offset_mapping, start_positions_orig, example_id are not expected by `RobertaAdapterModel.forward`,  you can safely ignore this message.
The following columns in the evaluation set don't have a corresponding argument in `RobertaAdapterModel.forward` and have been ignored: end_positions_orig, trigger_label, offset_mapping, start_positions_orig, example_id. If end_positions_orig, trigger_label, offset_mapping, start_positions_orig, example_id are not expected by `RobertaAdapterModel.forward`,  you can safely ignore this message.
The following columns in the evaluation set don't have a corresponding argument in `RobertaAdapterModel.forward` and have been ignored: end_positions_orig, trigger_label, offset_mapping, start_positions_orig,

Dataset: squad
{'epoch': 20.0,
 'eval_exact_match': 66.5,
 'eval_exact_match_p': 64.975,
 'eval_f1': 76.66626938339958,
 'eval_f1_p': 75.40635538449516,
 'eval_loss': -64.12259941101074,
 'eval_loss_amp': -1660.00916015625,
 'eval_loss_attn': 18.739916610717774,
 'eval_loss_cls': 1.5281682240962982}
